In [1]:
import sys
from pathlib import Path

workind_directory = Path.cwd().parent
if workind_directory not in sys.path:
    sys.path.append(str(workind_directory))

In [2]:
import json
import pandas as pd


with open("../data/raw/train.json") as f:
    data = json.load(f)

for essay in data:
    assert len(essay["tokens"]) == len(essay["trailing_whitespace"])
    assert len(essay["tokens"]) == len(essay["labels"])

del data

In [3]:
from pii_detection.probe_labels_utils import read_tokens_df

df = read_tokens_df("../data/train_shard.json")
df

,document,loc_in_doc,tokens,trailing_whitespace,labels
0,7,0,Design,True,O
1,7,1,Thinking,True,O
2,7,2,for,True,O
3,7,3,innovation,True,O
4,7,4,reflexion,False,O
...,...,...,...,...,...
3494932,22687,815,process,True,O
3494933,22687,816,explained,True,O
3494934,22687,817,above,False,O
3494935,22687,818,.,False,O


In [4]:
df.isna().sum()

document               0
loc_in_doc             0
tokens                 0
trailing_whitespace    0
labels                 0
dtype: int64

In [5]:
df["labels"].unique()

array(['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL',
       'B-EMAIL', 'B-ID_NUM', 'I-URL_PERSONAL', 'B-USERNAME',
       'B-PHONE_NUM', 'I-PHONE_NUM', 'B-STREET_ADDRESS',
       'I-STREET_ADDRESS', 'I-ID_NUM'], dtype=object)

In [6]:
df["labels"].value_counts()

labels
O                   3493006
B-NAME_STUDENT          978
I-NAME_STUDENT          740
B-URL_PERSONAL           84
B-ID_NUM                 50
B-EMAIL                  28
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64

In [7]:
# All emails contain "@" but not all "@" are emails

df[df["tokens"].str.contains("@")]["labels"].value_counts()

labels
B-EMAIL    28
O          11
Name: count, dtype: int64

In [8]:
rare_labels = set(df["labels"].unique()) - {"O", "B-NAME_STUDENT", "I-NAME_STUDENT"}
df_rare = df[df["labels"].isin(rare_labels)]

# to excel
try:
    df_rare.to_csv("../rare_labels.csv", index=False)
except PermissionError as e:
    print(e)
    print("Please close the file and try again")

df_rare.head(50)

,document,loc_in_doc,tokens,trailing_whitespace,labels
6402,317,611,https://www.jackson.com/list/explorehomepage.htm,False,B-URL_PERSONAL
11214,379,28,djones@gmail.com,True,B-EMAIL
11217,379,31,https://www.linkedin.com/in/mmartinez,True,B-URL_PERSONAL
13561,472,922,https://youtu.be/rFD2lJuvace,True,B-URL_PERSONAL
13720,609,26,860632713425,True,B-ID_NUM
13733,609,39,530670102508,True,B-ID_NUM
13744,609,50,530670102508,True,B-ID_NUM
13750,609,56,875673967537,True,B-ID_NUM
13756,609,62,860632713425,True,B-ID_NUM
13762,609,68,557349702179,True,B-ID_NUM


In [9]:
# Patterns
# Emails have "@" in them
# URLs have "." in them
# URLs have "http"/"www" in them
# 